In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import os
import sys
import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import font_manager
%matplotlib inline
import matplotlib.colors as colors


In [3]:
# resource configuration
rcParams['figure.dpi'] = 200

# fount information
rcParams['font.family'] = 'Equinor'
rcParams['font.size'] = 9

# fornt location
font_path = r'Equinor_regular'
font_files = font_manager.findSystemFonts(fontpaths=font_path)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

In [4]:
# where WellClass and Ga[ codes are located
sys.path.append('..')

In [5]:
# WellClass
# from well_class_v2 import csv_parser, Well
from src.WellClass.libs.utils import (
    csv_parser,
    yaml_parser,
)
from src.WellClass.libs.well_class import Well
from src.WellClass.libs.well_pressure import Pressure

## Some user options

In [6]:
# pick an example from given three options

case_type = 'cosmo'

# case_type = 'smeaheia_v1'

# case_type = 'smeaheia_v2'

In [7]:
# where pvt located
pvt_path = '../test_data/pvt_constants/'

# Examples

The following are the test examples.

In [8]:
# examples
smeaheia_v1 = {'well_input': r'GaP_input_Smeaheia_v3.csv', 
               'well_input_yaml': r'smeaheia.yaml', 
            #    'sim_path': r'/scratch/SCS/eim/SMEAHEIA', 
               'sim_path': r'../test_data/examples/smeaheia_v1',
               'simcase': r'GEN_NOLGR_PH2'}
smeaheia_v2 = {'well_input': r'GaP_input_Smeaheia_v3.csv', 
               'well_input_yaml': r'smeaheia.yaml', 
            #    'sim_path': r'/scratch/SCS/bkh/wbook/realization-0/iter-0/pflotran/model', 
               'sim_path': r'../test_data/examples/smeaheia_v2', 
               'simcase': r'TEMP-0'}
cosmo = {
         'well_input': r'GaP_input_Cosmo_v3.csv', 
         'well_input_yaml': r'cosmo.yaml', 
        #  'sim_path': r'/scratch/SCS/bkh/well_class_test1/realization-0/iter-0/pflotran/model', 
         'sim_path': r'../test_data/examples/cosmo', 
         'simcase': r'TEMP-0'}

In [9]:
examples = {
    'smeaheia_v1': smeaheia_v1,
    'smeaheia_v2': smeaheia_v2,
    'cosmo': cosmo
}

In [10]:
# the selected example for testing
case = examples[case_type]


In [11]:

# root_path = '/scratch/SCS/gpb/SCREEN/GaP_code'

sim_path = case['sim_path']


# Load well CSV file

Process CSV with well class.
Predefine a dictionary that includes the input CSV well file, the simulation path, and the PFT sim case name

In [12]:
# where well configuration file is located
well_name = os.path.join(sim_path, case['well_input'])

# load the well information
well_csv = csv_parser(well_name)

In [13]:
#Process well by running well class
my_well = Well( header       = well_csv['well_header'], 
                drilling     = well_csv['drilling'],
                casings      = well_csv['casing_cement'],
                geology      = well_csv['geology'],
                barriers     = well_csv['barriers'], 
                barrier_perm = well_csv['barrier_permeability'],
                co2_datum    = well_csv['co2_datum'],
           )

In [14]:
aa = pd.DataFrame(my_well.geology)
aa

,top_rkb,geol_unit,reservoir_flag,top_msl,base_msl
0,132,NORDLAND GP,False,105.0,517.0
1,544,HORDALAND GP,False,517.0,1014.0
2,1041,ROGALAND GP,False,1014.0,1136.0
3,1163,SHETLAND GP,False,1136.0,1417.0
4,1444,CROMER KNOLL GP,False,1417.0,2095.0
5,2122,VIKING GP,False,2095.0,2238.0
6,2265,JURASSIC SS,True,2238.0,2325.0
7,2352,SKAGERRAK FM,True,2325.0,2505.0
8,2532,SMITH BANK FM,True,2505.0,2638.0
9,2665,ZECHSTEIN GP,False,2638.0,3802.0


In [15]:
my_well.co2_datum

2370

In [16]:
well_csv['reservoir_pressure']

{'depth_msl': 2238, 'RP1': nan, 'RP2': '+ 20'}

In [17]:
my_pressure = Pressure(
    header      = well_csv['well_header'],
    reservoir_P = well_csv['reservoir_pressure'],
    co2_datum   = well_csv['co2_datum'],
    pvt_path    = pvt_path,
)

Hydrostatic pressure at reference depth 2238 is 218.89
RP1 set as hydrostatic P = 218.89 bar
RP2 is set as delta pressure, which yields P = 218.89 +20.00 = 238.89 bar
Reference depth: 2238


### CO2 Leakage

In [18]:
# pick one barrier
main_barrier = well_csv['main_barrier']
main_barrier

'cplug1'

In [19]:
# compute its leakage
barrier_leakage = my_pressure.compute_barrier_leakage(my_well, main_barrier)
barrier_leakage

barrier_perm:  {0: 0.01, 1: 10.0, 2: 1000.0}


,0.01,10.00,1000.00
RP1,0,0.00968,0.97926
RP2,0,0.01464,1.47506


# Load well Yaml file

Process yaml with well class.
Predefine a dictionary that includes the input yaml well file, the simulation path, and the PFT sim case name

In [20]:
# where well configuration file is located
well_name = os.path.join(sim_path, case['well_input_yaml'])

# # pydantic model
well_model = yaml_parser(well_name)
well_csv = json.loads(well_model.spec.model_dump_json())

In [21]:
#Process well by running well class
my_well_yaml = Well( header       = well_csv['well_header'], 
                     drilling     = well_csv['drilling'],
                     casings      = well_csv['casing_cement'],
                     geology      = well_csv['geology'],
                     barriers     = well_csv['barriers'], 
                     barrier_perm = well_csv['barrier_permeability'],
                     co2_datum    = well_csv['co2_datum'],
           )

In [22]:
bb = pd.DataFrame(my_well_yaml.geology)
bb

,top_rkb,geol_unit,reservoir_flag,top_msl,base_msl
0,132.0,NORDLAND GP,False,105.0,517.0
1,544.0,HORDALAND GP,False,517.0,1014.0
2,1041.0,ROGALAND GP,False,1014.0,1136.0
3,1163.0,SHETLAND GP,False,1136.0,1417.0
4,1444.0,CROMER KNOLL GP,False,1417.0,2095.0
5,2122.0,VIKING GP,False,2095.0,2238.0
6,2265.0,JURASSIC SS,True,2238.0,2325.0
7,2352.0,SKAGERRAK FM,True,2325.0,2505.0
8,2532.0,SMITH BANK FM,True,2505.0,2638.0
9,2665.0,ZECHSTEIN GP,False,2638.0,3802.0


In [23]:
my_well_yaml.co2_datum

2370.0

In [24]:
well_csv['reservoir_pressure']

{'depth_msl': 2238.0, 'RP1': None, 'RP2': '+ 20', 'RP3': None}

In [25]:
my_pressure_yaml = Pressure(
    header      = well_csv['well_header'],
    reservoir_P = well_csv['reservoir_pressure'],
    co2_datum   = well_csv['co2_datum'],
    pvt_path    = pvt_path,
)

Hydrostatic pressure at reference depth 2238 is 218.89
RP1 set as hydrostatic P = 218.89 bar
RP2 is set as delta pressure, which yields P = 218.89 +20.00 = 238.89 bar
None ignored
Reference depth: 2238.0


### CO2 Leakage

In [26]:
# pick one barrier
main_barrier = well_csv['main_barrier']
main_barrier

'cplug1'

In [27]:
# compute its leakage
barrier_leakage = my_pressure.compute_barrier_leakage(my_well, main_barrier)
barrier_leakage

barrier_perm:  {0: 0.01, 1: 10.0, 2: 1000.0}


,0.01,10.00,1000.00
RP1,0,0.00968,0.97926
RP2,0,0.01464,1.47506
